# The Netflix Formula: What Drives High Ratings and Viewer Engagement
**Emily Nguyen**

### Feature Engineering & Downloading Cleaned Dataset

In [1]:
import pandas as pd
# importing dataset and creating DataFrame
df = pd.read_csv("Netflix TV Shows and Movies.csv")
df.head()

,index,id,title,type,description,release_year,age_certification,runtime,imdb_id,imdb_score,imdb_votes
0,0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,tt0075314,8.3,795222.0
1,1,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,tt0071853,8.2,530877.0
2,2,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,tt0079470,8.0,392419.0
3,3,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,tt0070047,8.1,391942.0
4,4,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,tt0063929,8.8,72895.0


In [2]:
# get the number of rows and columns
num_rows, num_cols = df.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 5283
Number of columns: 11


### Data Cleaning & Preprocessing
---

**Handling Missing Values**

Before analyzing the dataset, I checked for missing values in each column. I was able to find that the 'age_certification' column had the most missing values, with 2,285 entries without an age rating. This is likely due to some older or international titles not having an assigned age rating. Given this is such a significant number, about 43% of the data, I decided to not drop the rows with missing age certification values since I could lose a lot of potentially useful information. 

Additionally, 'idmb_votes' had 16 missing values and 'description' had 5 missing values. All other columns, including 'title', 'type', 'release_year', 'runtime', and 'imdb_score', were complete. To handle these missing values:
- Titles without age_certification were kept in the dataset since other variables (like imdb_score and type) were still available for analysis.
- Missing imdb_votes or description entries were excluded from analyses.

In [3]:
# check for missing values in each column
df.isna().sum().sort_values(ascending=False)

age_certification    2285
imdb_votes             16
description             5
title                   0
id                      0
index                   0
type                    0
release_year            0
runtime                 0
imdb_id                 0
imdb_score              0
dtype: int64

In [4]:
# fill missing age_certification with 'Unknown'
df['age_certification'] = df['age_certification'].fillna('Unknown')

# drop rows where description and idmb_votes are missing
df = df.dropna(subset=['description', 'imdb_votes'])

# check for missing values in each column again
df.isna().sum().sort_values(ascending=False)


index                0
id                   0
title                0
type                 0
description          0
release_year         0
age_certification    0
runtime              0
imdb_id              0
imdb_score           0
imdb_votes           0
dtype: int64

**Data Type Conversions**

To ensure that each column in the dataset has an appropriate data type, I verified that each column had the appropriate data type, and confirmed that all the columns were correctly typed. 
- Numeric columns such as release_year, runtime, imdb_score, and imdb_votes were integer or float types to allow for arithmetic operations and plotting.
- Categorical or text-based columns, including title, type, description, age_certification, id, and imdb_id, were kept as objects (assuming strings).
- The index column is kept as an integer, serving as a unique identifier for rows in the dataset

In [5]:
df.dtypes

index                  int64
id                    object
title                 object
type                  object
description           object
release_year           int64
age_certification     object
runtime                int64
imdb_id               object
imdb_score           float64
imdb_votes           float64
dtype: object

**Filtering Unrealistic Runtimes**

During data cleaning, I also found that some titles had extremely short runtimes (0-10 minutes), which likely corresponds to data entry mistakes or trailers, shortclips, and previews rather than full episodes or movies. Including these entries could skew analyses related to popularity, ratings, and runtime. Given this, I filtered the dataset to include only titles with a runtime of 10 minutes or longer. 

In [6]:
# count rows with runtime less than 10
short_df = df[df['runtime'] < 10]
print(f"Number of titles with runtime under 10 minutes: {short_df.shape[0]}")

Number of titles with runtime under 10 minutes: 67


In [7]:
# remove rows with runtime less than 10
df = df[df['runtime'] >= 10]

num_rows, num_cols = df.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 5196
Number of columns: 11


After cleaning the dataset, the number of rows decreased from 5283 to **5196**, reflecting the removal of incomplete or unrealistic entries.

In [25]:
# feature engineered cols
df['content_age'] = 2025 - df['release_year']

df['content_decade'] = (df['release_year'] // 10) * 10

df['length_category'] = pd.cut(
    df['runtime'],
    bins=[0, 30, 60, 120, 180, 300],
    labels=['Short (<30)', 'Medium (30-60)', 'Feature (60-120)', 'Long (120-180)', 'Epic (180+)']
)

df['age_group'] = df['age_certification'].replace({
    'G': 'Family',
    'PG': 'Family',
    'PG-13': 'Teens',
    'R': 'Adults',
    'NC-17': 'Adults',
    None: 'Unknown'
})

age_map = {
    "TV-Y": "Kids",
    "TV-Y7": "Kids",
    "TV-G": "Kids",
    "G": "Kids",
    "PG": "Family/Teen",
    "TV-PG": "Family/Teen",
    "PG-13": "Family/Teen",
    "TV-14": "Family/Teen",
    "R": "Mature",
    "NC-17": "Mature",
    "TV-MA": "Mature",
}

df['age_group'] = df['age_certification'].map(age_map).fillna('Unknown')

df['rating_category'] = pd.cut(
    df['imdb_score'],
    bins=[0, 5, 7, 8.5, 10],
    labels=['Low (≤5)', 'Average (5–7)', 'Good (7–8.5)', 'Excellent (>8.5)']
)

In [27]:
# Save cleaned dataset to the same folder as this notebook/script
import os

# Get the current working directory (same folder where this notebook/script runs)
current_dir = os.getcwd()

# Define output file name
output_file = os.path.join(current_dir, "Netflix_Cleaned_Nov_3_2025.csv")

# Save the cleaned dataset to CSV
df.to_csv(output_file, index=False)

print(f"Cleaned dataset saved successfully to: {output_file}")


Cleaned dataset saved successfully to: /Users/emilynguyen/Desktop/itcs4122/NetflixRatingsAnalysis/Netflix_Cleaned_Nov_3_2025.csv
